# FMU ME: Bouncing Ball

This example demonstrates **Model Exchange FMU** integration with PathSim. Unlike co-simulation FMUs, Model Exchange FMUs provide only the differential equations. PathSim's solvers perform the numerical integration and event detection.

You can also find the FMU integration tests in the [GitHub repository](https://github.com/milanofthe/pathsim/blob/master/tests/evals/).

The bouncing ball combines continuous dynamics with discrete events:

$$\frac{dh}{dt} = v$$
$$\frac{dv}{dt} = g$$

where $h$ is height, $v$ is velocity, and $g = -9.81\,\text{m/s}^2$. At impact ($h = 0$), velocity reverses with energy loss: $v^+ = -e \cdot v^-$ where $e \in [0,1]$ is the restitution coefficient.

## Import and Setup

Note that FMPy must be installed to use FMU blocks:

```bash
pip install fmpy
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Apply PathSim docs matplotlib style for consistent, theme-friendly figures
plt.style.use('../pathsim_docs.mplstyle')

from pathlib import Path
from pathsim import Simulation, Connection
from pathsim.blocks import ModelExchangeFMU, Scope
from pathsim.solvers import RKBS32

## FMU Path

The FMU contains binaries for multiple platforms (Windows, Linux, macOS):

In [ ]:
notebook_dir = Path().resolve()
fmu_path = notebook_dir / "data" / "BouncingBall_ME.fmu"

# Verify FMU exists
if not fmu_path.exists():
    raise FileNotFoundError(f"FMU file not found at {fmu_path}")

## System Definition

In [ ]:
# Create the Model Exchange FMU block
fmu = ModelExchangeFMU(
    fmu_path=str(fmu_path),
    instance_name="bouncing_ball",
    start_values={"e": 0.7},  # coefficient of restitution
    tolerance=1e-10,
    verbose=False
)

# Scope to record height and velocity
sco = Scope(labels=["h [m]", "v [m/s]"])

blocks = [fmu, sco]

# Connect FMU outputs to scope
connections = [
    Connection(fmu[0], sco[0]),  # height
    Connection(fmu[1], sco[1]),  # velocity
]

Display FMU metadata:

In [ ]:
# Display FMU metadata (accessed via fmu_wrapper)
md = fmu.fmu_wrapper.model_description
print(f"Model Name: {md.modelName}")
print(f"FMI Version: {fmu.fmu_wrapper.fmi_version}")
print(f"Description: {md.description}")
print(f"Generation Tool: {md.generationTool}")
print(f"\nContinuous states: {fmu.fmu_wrapper.n_states}")
print(f"Event indicators: {fmu.fmu_wrapper.n_event_indicators}")
print(f"Outputs: {len(fmu.fmu_wrapper.output_refs)}")

## Simulation Setup

In [ ]:
# Initialize simulation
sim = Simulation(
    blocks,
    connections,
    dt=0.01,
    dt_max=0.01,
    Solver=RKBS32,
    tolerance_lte_rel=1e-6,
    tolerance_lte_abs=1e-9,
    log=True
)

# Run simulation
sim.run(4.0)

## Results

Plot the trajectory:

In [ ]:
sco.plot()
plt.show()

## Event Visualization

Mark the detected bounce events:

In [ ]:
time, (h, v) = sco.read()

fig, axes = plt.subplots(2, 1, figsize=(9, 6), sharex=True, dpi=130)

# Mark bounce events
for ax in axes:
    for t in fmu.events[0]:
        ax.axvline(t, ls="--", lw=1, c="gray", 
                   label="Bounce Event" if t == list(fmu.events[0])[0] else None)

axes[0].plot(time, h, lw=2)
axes[0].set_ylabel("Height [m]")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(time, v, lw=2, color='orange')
axes[1].set_xlabel("Time [s]")
axes[1].set_ylabel("Velocity [m/s]")
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()